In [1]:
# jupyter notebook uses the 'tx-bridge' environment

import os
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
import pdal
import json

In [2]:
int_buffer = 300
int_tile_x = 5000
int_tile_y = 5000
int_overlap = 150

In [3]:
str_input_polygon = r'C:\test\bridge_sanantonio\aoi_bridge_sa_test_ar_3857.shp'
str_copc_source = r'D:\globus_transfer\bridge_merge_pecos_dallas\bridge_merge_pecos_dallas.copc.laz'
STR_OUTPUT_PATH = r'D:\globus_transfer\01_dallas_lidar_tiles'

In [4]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
def fn_create_tiles_gdf (str_aoi_shp_path,
                         int_buffer,
                         int_tile_x,
                         int_tile_y,
                         int_overlap):
    
    
    # define the "lambert" espg
    str_lambert = "epsg:3857"
    
    # read the "area of interest" shapefile in to geopandas dataframe
    gdf_aoi_prj = gpd.read_file(str_aoi_shp_path)
    
    # convert the input shapefile to lambert
    gdf_aoi_lambert = gdf_aoi_prj.to_crs(str_lambert)
    
    #buffer the polygons in the input shapefile
    gdf_aoi_lambert['geometry'] = gdf_aoi_lambert.geometry.buffer(int_buffer)
    
    
    for index_gdf_int, row_gdf_int in gdf_aoi_lambert.iterrows():
    
        # the geometry from the requested polygon as wellKnownText
        boundary_geom_WKT = gdf_aoi_lambert['geometry'][index_gdf_int]  # to WellKnownText
    
        # create geodataframe of just the current row
        gdf_aoi_lambert_current = gpd.GeoDataFrame(gdf_aoi_lambert.iloc[[index_gdf_int]])
    
        # reset the index
        gdf_aoi_lambert_current = gdf_aoi_lambert_current.reset_index(drop=True)
        
        # get the geodataframe coordiante system
        gdf_aoi_lambert_current = gdf_aoi_lambert_current.set_crs(str_lambert)
    
        # the bounding box of the requested lambert polygon
        b = boundary_geom_WKT.bounds
    
        # convert the bounding coordinates to integers
        list_int_b = []
        for i in b:
            list_int_b.append(int(i//1))
    
        # determine the width and height of the requested polygon
        flt_delta_x = list_int_b[2] - list_int_b[0]
        flt_delta_y = list_int_b[3] - list_int_b[1]
    
        # determine the number of tiles in the x and y direction
        int_tiles_in_x = (flt_delta_x // (int_tile_x - int_overlap)) + 1
        int_tiles_in_y = (flt_delta_y // (int_tile_y - int_overlap)) + 1
    
        list_tile_name = []
        list_geometry = []
    
        list_point_x = []
        list_point_y = []
    
        for value_x in range(int_tiles_in_x):
            list_point_x = []
            int_current_start_x = (value_x * (int_tile_x - int_overlap)) + list_int_b[0]
            list_point_x = [int_current_start_x, 
                            int_current_start_x + int_tile_x, 
                            int_current_start_x + int_tile_x, 
                            int_current_start_x,
                            int_current_start_x]
    
            for value_y in range(int_tiles_in_y):
                list_point_y = []
                int_current_start_y = (value_y * (int_tile_y - int_overlap)) + list_int_b[1] 
                list_point_y = [int_current_start_y, 
                                int_current_start_y,
                                int_current_start_y + int_tile_y,
                                int_current_start_y + int_tile_y,
                                int_current_start_y]
    
                polygon_geom = Polygon(zip(list_point_x, list_point_y))
                list_geometry.append(polygon_geom)
    
                str_time_name = str(value_x) + '_' + str(value_y)
                list_tile_name.append(str_time_name)
    
    # create a pandas dataframe
    df = pd.DataFrame({'tile_name': list_tile_name, 'geometry': list_geometry})
    
    # convert the pandas dataframe to a geopandas dataframe
    gdf_tiles = gpd.GeoDataFrame(df, geometry='geometry')
    
    # set the tile footprint crs
    gdf_tiles = gdf_tiles.set_crs(str_lambert)
    
    # intersect the tiles and the requested polygon
    gdf_intersected_tiles = gpd.overlay(gdf_tiles, gdf_aoi_lambert_current, how='intersection')
    
    # get a unique list of the intersected tiles
    arr_tiles_intersect = gdf_intersected_tiles['tile_name'].unique()
    
    # convert the array to a list
    list_tiles_intersect = arr_tiles_intersect.tolist()
    
    # new geodataframe of the tiles intersected (but not clipped)
    gdf_tiles_intersect_only = gdf_tiles[gdf_tiles['tile_name'].isin(list_tiles_intersect)]
    
    # reset the remaining tile index
    gdf_tiles_intersect_only = gdf_tiles_intersect_only.reset_index(drop=True)
    
    return gdf_tiles_intersect_only
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [5]:
gdf_tiles = fn_create_tiles_gdf(str_input_polygon,
                                int_buffer,
                                int_tile_x,
                                int_tile_y,
                                int_overlap)

In [6]:
gdf_tiles.explore()

In [7]:
len(gdf_tiles)

1050

In [8]:
%%time

for index, row in gdf_tiles.iterrows():
    str_tile_name = gdf_tiles.loc[index]['tile_name']
    INT_CLASS = 17
    
    b = gdf_tiles.loc[index]['geometry'].bounds #the bounding box of the requested lambert polygon
    str_classification = "Classification[" + str(INT_CLASS) + ":" + str(INT_CLASS) + "]"
    str_las = os.path.join(STR_OUTPUT_PATH, str_tile_name + '_class_' + str(INT_CLASS) + '.las')
    
    pipeline_class_las = {
        "pipeline": [
            {   
                'bounds':str(([b[0], b[2]],[b[1], b[3]])),
                "filename":str_copc_source,
                "type":"readers.copc",
                "tag":"readdata"
            },
            {   
                "type":"filters.range",
                "limits": str_classification,
                "tag":"class_points"
            },
            {
                "filename": str_las,
                "inputs": [ "class_points" ],
                "type": "writers.las"
            }
        ]}
    
    #execute the pdal pipeline
    pipeline = pdal.Pipeline(json.dumps(pipeline_class_las))
    n_points = pipeline.execute()

CPU times: total: 3min 48s
Wall time: 1min 40s
